In [3]:
from main.compiling.compilers.AncillaPerCheckCompiler import AncillaPerCheckCompiler
from main.compiling.noise.models.CircuitLevelNoise import CircuitLevelNoise
from main.compiling.noise.models.superconducting_inspired_noise import SuperconductingInspired
from main.compiling.syndrome_extraction.extractors.ancilla_per_check.mixed.CxCyCzExtractor import CxCyCzExtractor
from main.codes.tic_tac_toe.HoneycombCode import HoneycombCode
from main.codes.tic_tac_toe.gauge_honeycomb_code import GaugeHoneycombCode
from main.codes.tic_tac_toe.gauge_floquet_colour_code import GaugeFloquetColourCode
from main.codes.tic_tac_toe.FloquetColourCode import FloquetColourCode
from main.codes.tic_tac_toe.TicTacToeCode import TicTacToeCode
from main.utils.enums import State
from main.building_blocks.pauli import Pauli
from main.building_blocks.pauli.PauliLetter import PauliLetter
import stim
import sinter 
import matplotlib.pyplot as plt
import numpy as np
from beliefmatching import BeliefMatchingSinterDecoder
from main.utils.Colour import Blue, Green, Red
import hashlib

from main.building_blocks.detectors.Stabilizer import Stabilizer
from pathlib import Path

class HoneycombCodeZ(TicTacToeCode):
    """Adaptation of the Honeycomb code where the measurement order has been permuted
    """
    def __init__(self, distance: int):
        tic_tac_toe = [
                (Blue, PauliLetter('Z')),
                (Red, PauliLetter('X')),
                (Green, PauliLetter('Y'))]

        super().__init__(distance, tic_tac_toe)

# load the data.
# seperate running and loading.



# Help functions

In [11]:



def calculate_threshold(code_name, distances, observable_index, layer_factor,pers,noise_model):
    tasks = []
    if code_name == "HoneycombCodeX":
        code = HoneycombCode
        measurement_rounds = 3
    elif code_name == "HoneycombCodeZ":
        code = HoneycombCodeZ
        measurement_rounds = 3
    elif code_name == "GaugeHoneycombCodeX":
        code = GaugeHoneycombCode
        gauge_factor = 2
        measurement_rounds = 6
    elif code_name == "FloquetColourCodeX":
        code = FloquetColourCode
        measurement_rounds = 6
    elif code_name == "GaugeFloquetColourCodeX":
        code = GaugeFloquetColourCode
        gauge_factor = 2
        measurement_rounds = 12


    for d in distances:
        for index, p in enumerate(pers):

            # generate circuit
#            if code_name[-1] == 'X':
            if code_name in ["GaugeHoneycombCodeX", "GaugeFloquetColourCodeX", "GaugeFloquetColourCodeZ", "GaugeHoneycombCodeZ"]:
                circuit = load_or_create_stim_circuit(code(d,gauge_factor), observable_index,layer_factor*d, p, code_name[-1], noise_model)
            else:
                circuit = load_or_create_stim_circuit(code(d), observable_index,layer_factor*d, p, code_name[-1], noise_model)
 #           elif code_name[-1] == 'Z':
  #              circuit = get_Z_stim_circuit(code(d, gauge_factor), observable_index, layer_factor*d, p)
            tasks.append(sinter.Task(
                    circuit=circuit,
                    detector_error_model=circuit.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True, ignore_decomposition_failures=True),
                    json_metadata={
                        'code': code_name,
                        'distance': d,
                        'p': p,
                        'layers': layer_factor*d,
                        'r': layer_factor,
                        'measurement_rounds': measurement_rounds,
                        'total_measurement_rounds': d * layer_factor * measurement_rounds,
                        'noise_model': noise_model}))

                        

    samples = sinter.collect(
        tasks=tasks,
        hint_num_tasks=len(tasks),
        num_workers=2,
        max_shots=1000000,
        max_errors=100,
        decoders=['pymatching'],
        custom_decoders={'beliefmatching': BeliefMatchingSinterDecoder()},
        print_progress=True,
        save_resume_filepath=f'./resume_15_2/data.csv',)
    
    # Print samples as CSV data.
    
    return(samples)

def load_or_create_stim_circuit(code, observable_index, layers, p, pauli_type, noise_model):
    hash_fields = (type(code).__name__, code.distance, observable_index, layers, p,pauli_type, noise_model)
    hash_input = str(hash_fields).encode('UTF-8')
    hashed = int(hashlib.md5(hash_input).hexdigest(), 16)
    filepath = '../stim_circuits/'
    filepath = Path(f"../stim_circuits/{hashed}.stim")
    if filepath.is_file():
        stim_circuit = stim.Circuit.from_file(filepath)
    else:
        if pauli_type == 'X':
            stim_circuit = get_X_stim_circuit(code, observable_index, layers, p, noise_model)
        elif pauli_type == 'Z':
            stim_circuit = get_Z_stim_circuit(code, observable_index, layers, p)
        stim_circuit.to_file(filepath)
    return stim_circuit


def get_X_stim_circuit(code,observable_index,layers,p,noise_model):

    if noise_model == "CircuitLevelNoise":
        compiler = AncillaPerCheckCompiler(CircuitLevelNoise(p,p,p,p,p), syndrome_extractor=CxCyCzExtractor())
    elif noise_model == "SuperconductingNoise":
        compiler = AncillaPerCheckCompiler(SuperconductingInspired(p), syndrome_extractor=CxCyCzExtractor())
    data_qubits = code.data_qubits.values()
    data_qubit_initial_states = {qubit: State.Plus for qubit in data_qubits}
    final_measurements = [Pauli(qubit, PauliLetter('X')) for qubit in data_qubits]
    logical_observables = [code.logical_qubits[observable_index].x]#, code.logical_qubits[1].x]

    initial_stabilizers = []
    for check in code.check_schedule[0]:
        initial_stabilizers.append(Stabilizer([(0,check)],0))
    stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers, initial_stabilizers=initial_stabilizers,final_measurements=final_measurements,logical_observables=logical_observables)
    return(stim_circuit)

def get_Z_stim_circuit(code,observable_index,layers,p,noise_model):
    if noise_model == "CircuitLevelNoise":
        compiler = AncillaPerCheckCompiler(CircuitLevelNoise(p,p,p,p,p), syndrome_extractor=CxCyCzExtractor())
    elif noise_model == "SuperconductingNoise":
        compiler = AncillaPerCheckCompiler(SuperconductingInspired(p), syndrome_extractor=CxCyCzExtractor())
    data_qubits = code.data_qubits.values()
    data_qubit_initial_states = {qubit: State.Zero for qubit in data_qubits}
    final_measurements = [Pauli(qubit, PauliLetter('Z')) for qubit in data_qubits]
    logical_observables = [code.logical_qubits[observable_index].z]
    stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers,initial_states= data_qubit_initial_states, final_measurements=final_measurements,logical_observables=logical_observables)
    return(stim_circuit)

In [12]:
#gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[4,8], 1,2, [0.00009],'CircuitLevelNoise')
#gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[4, 8], 1,4, [0.00009],'CircuitLevelNoise')
#fcc_X1_samples = calculate_threshold('FloquetColourCodeX',[4,8], 1,4, [0.00009],'CircuitLevelNoise')
#hcc_X1_samples = calculate_threshold('HoneycombCodeX',[4,8], 1,8, [0.00009],'CircuitLevelNoise')
gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[16], 1,2, [0.00075, 0.001, 0.002, 0.003], 'CircuitLevelNoise')
gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[16], 1,4, [0.00075, 0.001, 0.002, 0.003],'CircuitLevelNoise')
fcc_X1_samples = calculate_threshold('FloquetColourCodeX',[16], 1,4, [0.00075, 0.001, 0.002, 0.003], 'CircuitLevelNoise')
hcc_X1_samples = calculate_threshold('HoneycombCodeX',[16], 1,8, [0.00075, 0.001, 0.002, 0.003], 'CircuitLevelNoise')

Starting workers...
Finding work...
1 cases left:
    pymatching processes=1  ~core_mins_left=2       shots_left=500000   errors_left=100     {code=GaugeFloquetColourCodeX,distance=8,layers=16,measurement_rounds=12,noise_model=CircuitLevelNoise,p=9e-05,r=2,total_measurement_rounds=192}
Done collecting
Starting workers...
Finding work...
1 cases left:
    pymatching processes=1  ~core_mins_left=2       shots_left=500000   errors_left=100     {code=GaugeHoneycombCodeX,distance=8,layers=32,measurement_rounds=6,noise_model=CircuitLevelNoise,p=9e-05,r=4,total_measurement_rounds=192}
Done collecting
